In [1]:
import random
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor

path = os.getcwd()
file_path = os.path.join(path,"file")
sub_path = os.path.join(path,"제출")
train_df = pd.read_csv(os.path.join(file_path,"train.csv"))
test_df = pd.read_csv(os.path.join(file_path,"test.csv"))

In [2]:
display(train_df.head())

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0


In [3]:
#시계열 특성을 학습에 반영하기 위해 timestamp를 월, 일, 시간으로 나눕니다
train_df['year'] = train_df['timestamp'].apply(lambda x : int(x[0:4]))
train_df['month'] = train_df['timestamp'].apply(lambda x : int(x[5:7]))
train_df['day'] = train_df['timestamp'].apply(lambda x : int(x[8:10]))

test_df['year'] = test_df['timestamp'].apply(lambda x : int(x[0:4]))
test_df['month'] = test_df['timestamp'].apply(lambda x : int(x[5:7]))
test_df['day'] = test_df['timestamp'].apply(lambda x : int(x[8:10]))

In [4]:
from pytimekr import pytimekr
import datetime

List_holiday = []
for i in range(5,1551,7):
    List_holiday.append((datetime.datetime(2019, 1, 1)+datetime.timedelta(i)).strftime("%Y-%m-%d"))
    
for i in range(2019,2024):
    List_holiday.extend(pytimekr.holidays(year=i)) #리스트 형태로 반환

for i in range(len(List_holiday)):
    List_holiday[i] = str(List_holiday[i])
np.sort(List_holiday)

array(['2019-01-01', '2019-01-06', '2019-01-13', '2019-01-20',
       '2019-01-27', '2019-02-03', '2019-02-04', '2019-02-05',
       '2019-02-06', '2019-02-10', '2019-02-17', '2019-02-24',
       '2019-03-01', '2019-03-03', '2019-03-10', '2019-03-17',
       '2019-03-24', '2019-03-31', '2019-04-07', '2019-04-14',
       '2019-04-21', '2019-04-28', '2019-05-05', '2019-05-05',
       '2019-05-12', '2019-05-12', '2019-05-19', '2019-05-26',
       '2019-06-02', '2019-06-06', '2019-06-09', '2019-06-16',
       '2019-06-23', '2019-06-30', '2019-07-07', '2019-07-14',
       '2019-07-21', '2019-07-28', '2019-08-04', '2019-08-11',
       '2019-08-15', '2019-08-18', '2019-08-25', '2019-09-01',
       '2019-09-08', '2019-09-12', '2019-09-13', '2019-09-14',
       '2019-09-15', '2019-09-22', '2019-09-29', '2019-10-03',
       '2019-10-06', '2019-10-09', '2019-10-13', '2019-10-20',
       '2019-10-27', '2019-11-03', '2019-11-10', '2019-11-17',
       '2019-11-24', '2019-12-01', '2019-12-08', '2019-

In [5]:
train_df.loc[train_df['timestamp'].isin(List_holiday)==True,'Holiday']=0
train_df.loc[train_df['timestamp'].isin(List_holiday)==False,'Holiday']=1
test_df.loc[test_df['timestamp'].isin(List_holiday)==True,'Holiday']=0
test_df.loc[test_df['timestamp'].isin(List_holiday)==False,'Holiday']=1

In [6]:
train_x = train_df.drop(columns=['ID', 'timestamp', 'supply(kg)', 'price(원/kg)'])
train_y = train_df['price(원/kg)']

test_x = test_df.drop(columns=['ID', 'timestamp'])

In [7]:
qual_col = ['item', 'corporation', 'location']

for i in qual_col:
    le = LabelEncoder()
    train_x[i]=le.fit_transform(train_x[i])
    test_x[i]=le.transform(test_x[i]) #test 데이터에 대해서 fit하는 것은 data leakage에 해당합니다

print('Done.')

Done.


In [8]:
# def minmaxscaler(data,Min=[],Max=[]):
#     if len(Min)==0 and len(Max)==0:
#         return (data-np.min(data,axis=0))/(np.max(data,axis=0)-np.min(data,axis=0)), np.min(data,axis=0),np.max(data,axis=0)
#     else:
#         return (data-Min)/(Max-Min)
# def standardscaler(data,Mean=[],Std=[]):
#     if len(Mean)==0 and len(Std)==0:
#         return (data-np.mean(data,axis=0))/np.std(data,axis=0), np.mean(data,axis=0),np.std(data,axis=0)
#     else:
#         return (data - Mean) / Std
# re_train_x,Mean_train,Std_train = standardscaler(train_x)
# re_test_x = standardscaler(test_x,Mean_train,Std_train)

In [9]:
model = CatBoostRegressor(random_state=0, max_depth=16 ,n_estimators=2000)
model.fit(train_x, train_y)

Learning rate set to 0.044432
0:	learn: 1965.8169371	total: 182ms	remaining: 6m 3s
1:	learn: 1904.4713541	total: 297ms	remaining: 4m 56s
2:	learn: 1850.0127950	total: 301ms	remaining: 3m 20s
3:	learn: 1795.2606766	total: 310ms	remaining: 2m 34s
4:	learn: 1743.2329941	total: 423ms	remaining: 2m 48s
5:	learn: 1700.1560955	total: 426ms	remaining: 2m 21s
6:	learn: 1653.9665037	total: 540ms	remaining: 2m 33s
7:	learn: 1608.8093894	total: 653ms	remaining: 2m 42s
8:	learn: 1566.1504346	total: 742ms	remaining: 2m 44s
9:	learn: 1526.6989474	total: 881ms	remaining: 2m 55s
10:	learn: 1489.0733206	total: 994ms	remaining: 2m 59s
11:	learn: 1453.3341649	total: 1.1s	remaining: 3m 2s
12:	learn: 1419.0111503	total: 1.22s	remaining: 3m 5s
13:	learn: 1386.7026641	total: 1.32s	remaining: 3m 7s
14:	learn: 1360.5057128	total: 1.33s	remaining: 2m 55s
15:	learn: 1331.8488005	total: 1.44s	remaining: 2m 58s
16:	learn: 1304.4743195	total: 1.57s	remaining: 3m 3s
17:	learn: 1279.5461040	total: 1.69s	remaining: 3m 

152:	learn: 778.0919915	total: 16.1s	remaining: 3m 14s
153:	learn: 777.4879304	total: 16.2s	remaining: 3m 14s
154:	learn: 776.5876715	total: 16.4s	remaining: 3m 14s
155:	learn: 775.9594698	total: 16.5s	remaining: 3m 14s
156:	learn: 775.3609939	total: 16.6s	remaining: 3m 14s
157:	learn: 774.4281621	total: 16.7s	remaining: 3m 14s
158:	learn: 773.7268306	total: 16.8s	remaining: 3m 15s
159:	learn: 772.6256944	total: 17s	remaining: 3m 15s
160:	learn: 771.9409527	total: 17.1s	remaining: 3m 14s
161:	learn: 771.5155574	total: 17.2s	remaining: 3m 14s
162:	learn: 771.5028340	total: 17.2s	remaining: 3m 13s
163:	learn: 770.7876845	total: 17.3s	remaining: 3m 13s
164:	learn: 770.1704494	total: 17.4s	remaining: 3m 13s
165:	learn: 769.8263101	total: 17.5s	remaining: 3m 13s
166:	learn: 768.8518119	total: 17.6s	remaining: 3m 13s
167:	learn: 768.0441542	total: 17.8s	remaining: 3m 13s
168:	learn: 768.0422518	total: 17.8s	remaining: 3m 12s
169:	learn: 767.6803923	total: 17.9s	remaining: 3m 12s
170:	learn: 

305:	learn: 689.5167406	total: 33.2s	remaining: 3m 3s
306:	learn: 689.2942281	total: 33.3s	remaining: 3m 3s
307:	learn: 688.9635899	total: 33.4s	remaining: 3m 3s
308:	learn: 688.5333450	total: 33.5s	remaining: 3m 3s
309:	learn: 688.2369880	total: 33.7s	remaining: 3m 3s
310:	learn: 687.4698810	total: 33.8s	remaining: 3m 3s
311:	learn: 686.8402674	total: 33.9s	remaining: 3m 3s
312:	learn: 686.2658351	total: 34s	remaining: 3m 3s
313:	learn: 685.6362182	total: 34.1s	remaining: 3m 3s
314:	learn: 685.0905450	total: 34.2s	remaining: 3m 3s
315:	learn: 684.5953052	total: 34.4s	remaining: 3m 3s
316:	learn: 684.2271659	total: 34.5s	remaining: 3m 3s
317:	learn: 683.9589771	total: 34.6s	remaining: 3m 3s
318:	learn: 683.6688150	total: 34.7s	remaining: 3m 3s
319:	learn: 683.0570132	total: 34.9s	remaining: 3m 2s
320:	learn: 682.6848827	total: 35s	remaining: 3m 2s
321:	learn: 682.4243210	total: 35.1s	remaining: 3m 2s
322:	learn: 681.9262349	total: 35.2s	remaining: 3m 2s
323:	learn: 681.3843853	total: 3

457:	learn: 627.4784014	total: 51s	remaining: 2m 51s
458:	learn: 627.2700760	total: 51.1s	remaining: 2m 51s
459:	learn: 627.0553851	total: 51.3s	remaining: 2m 51s
460:	learn: 626.6063990	total: 51.4s	remaining: 2m 51s
461:	learn: 626.1943602	total: 51.5s	remaining: 2m 51s
462:	learn: 625.8370371	total: 51.6s	remaining: 2m 51s
463:	learn: 625.6547375	total: 51.7s	remaining: 2m 51s
464:	learn: 625.2393771	total: 51.8s	remaining: 2m 51s
465:	learn: 624.8318101	total: 52s	remaining: 2m 51s
466:	learn: 624.4169285	total: 52.1s	remaining: 2m 50s
467:	learn: 623.9060421	total: 52.2s	remaining: 2m 50s
468:	learn: 623.4948372	total: 52.3s	remaining: 2m 50s
469:	learn: 622.8534354	total: 52.4s	remaining: 2m 50s
470:	learn: 622.4677875	total: 52.5s	remaining: 2m 50s
471:	learn: 622.0777675	total: 52.7s	remaining: 2m 50s
472:	learn: 621.7141713	total: 52.8s	remaining: 2m 50s
473:	learn: 621.5366188	total: 52.9s	remaining: 2m 50s
474:	learn: 621.1356297	total: 53s	remaining: 2m 50s
475:	learn: 620.

607:	learn: 573.9508993	total: 1m 8s	remaining: 2m 37s
608:	learn: 573.6085652	total: 1m 8s	remaining: 2m 37s
609:	learn: 573.3385261	total: 1m 8s	remaining: 2m 36s
610:	learn: 572.8998562	total: 1m 8s	remaining: 2m 36s
611:	learn: 572.6891259	total: 1m 9s	remaining: 2m 36s
612:	learn: 572.5346393	total: 1m 9s	remaining: 2m 36s
613:	learn: 572.1314006	total: 1m 9s	remaining: 2m 36s
614:	learn: 571.9432914	total: 1m 9s	remaining: 2m 36s
615:	learn: 571.7630067	total: 1m 9s	remaining: 2m 36s
616:	learn: 571.4524055	total: 1m 9s	remaining: 2m 36s
617:	learn: 571.1279456	total: 1m 9s	remaining: 2m 36s
618:	learn: 570.9206594	total: 1m 9s	remaining: 2m 35s
619:	learn: 570.5857110	total: 1m 10s	remaining: 2m 35s
620:	learn: 570.2071071	total: 1m 10s	remaining: 2m 35s
621:	learn: 569.9137208	total: 1m 10s	remaining: 2m 35s
622:	learn: 569.5175055	total: 1m 10s	remaining: 2m 35s
623:	learn: 569.1506685	total: 1m 10s	remaining: 2m 35s
624:	learn: 568.7972521	total: 1m 10s	remaining: 2m 35s
625:

...

1341:	learn: 418.8084105	total: 2m 39s	remaining: 1m 18s
1342:	learn: 418.6505153	total: 2m 39s	remaining: 1m 17s
1343:	learn: 418.5499009	total: 2m 39s	remaining: 1m 17s
1344:	learn: 418.4230085	total: 2m 39s	remaining: 1m 17s
1345:	learn: 418.2496227	total: 2m 39s	remaining: 1m 17s
1346:	learn: 418.0634877	total: 2m 39s	remaining: 1m 17s
1347:	learn: 417.9596274	total: 2m 39s	remaining: 1m 17s
1348:	learn: 417.7667890	total: 2m 40s	remaining: 1m 17s
1349:	learn: 417.6546356	total: 2m 40s	remaining: 1m 17s
1350:	learn: 417.4540955	total: 2m 40s	remaining: 1m 17s
1351:	learn: 417.3342462	total: 2m 40s	remaining: 1m 16s
1352:	learn: 417.2072098	total: 2m 40s	remaining: 1m 16s
1353:	learn: 416.9844512	total: 2m 40s	remaining: 1m 16s
1354:	learn: 416.9160420	total: 2m 40s	remaining: 1m 16s
1355:	learn: 416.8318642	total: 2m 40s	remaining: 1m 16s
1356:	learn: 416.6728987	total: 2m 41s	remaining: 1m 16s
1357:	learn: 416.5906641	total: 2m 41s	remaining: 1m 16s
1358:	learn: 416.3987356	total:

1487:	learn: 398.7716024	total: 2m 57s	remaining: 1m 1s
1488:	learn: 398.5787040	total: 2m 57s	remaining: 1m 1s
1489:	learn: 398.5189436	total: 2m 58s	remaining: 1m
1490:	learn: 398.4448853	total: 2m 58s	remaining: 1m
1491:	learn: 398.2922603	total: 2m 58s	remaining: 1m
1492:	learn: 398.2334260	total: 2m 58s	remaining: 1m
1493:	learn: 398.1231469	total: 2m 58s	remaining: 1m
1494:	learn: 397.9563817	total: 2m 58s	remaining: 1m
1495:	learn: 397.7879402	total: 2m 58s	remaining: 1m
1496:	learn: 397.6682591	total: 2m 58s	remaining: 1m
1497:	learn: 397.5060932	total: 2m 59s	remaining: 1m
1498:	learn: 397.3609184	total: 2m 59s	remaining: 59.9s
1499:	learn: 397.2203596	total: 2m 59s	remaining: 59.8s
1500:	learn: 397.1475886	total: 2m 59s	remaining: 59.7s
1501:	learn: 396.9190326	total: 2m 59s	remaining: 59.5s
1502:	learn: 396.7998313	total: 2m 59s	remaining: 59.4s
1503:	learn: 396.6455459	total: 2m 59s	remaining: 59.3s
1504:	learn: 396.4547311	total: 2m 59s	remaining: 59.2s
1505:	learn: 396.41

1637:	learn: 379.0097574	total: 3m 16s	remaining: 43.5s
1638:	learn: 378.9479852	total: 3m 16s	remaining: 43.4s
1639:	learn: 378.8884943	total: 3m 17s	remaining: 43.3s
1640:	learn: 378.7803077	total: 3m 17s	remaining: 43.1s
1641:	learn: 378.7197389	total: 3m 17s	remaining: 43s
1642:	learn: 378.6037220	total: 3m 17s	remaining: 42.9s
1643:	learn: 378.4882420	total: 3m 17s	remaining: 42.8s
1644:	learn: 378.4252813	total: 3m 17s	remaining: 42.7s
1645:	learn: 378.2079388	total: 3m 17s	remaining: 42.5s
1646:	learn: 378.0999416	total: 3m 17s	remaining: 42.4s
1647:	learn: 377.9687997	total: 3m 18s	remaining: 42.3s
1648:	learn: 377.8383462	total: 3m 18s	remaining: 42.2s
1649:	learn: 377.7279330	total: 3m 18s	remaining: 42.1s
1650:	learn: 377.6807122	total: 3m 18s	remaining: 41.9s
1651:	learn: 377.5362664	total: 3m 18s	remaining: 41.8s
1652:	learn: 377.4185019	total: 3m 18s	remaining: 41.7s
1653:	learn: 377.2993817	total: 3m 18s	remaining: 41.6s
1654:	learn: 377.1944690	total: 3m 18s	remaining: 

1785:	learn: 360.9708589	total: 3m 35s	remaining: 25.8s
1786:	learn: 360.9121228	total: 3m 35s	remaining: 25.7s
1787:	learn: 360.6974207	total: 3m 35s	remaining: 25.6s
1788:	learn: 360.5957259	total: 3m 36s	remaining: 25.5s
1789:	learn: 360.5323311	total: 3m 36s	remaining: 25.4s
1790:	learn: 360.3550227	total: 3m 36s	remaining: 25.2s
1791:	learn: 360.2122160	total: 3m 36s	remaining: 25.1s
1792:	learn: 360.0831139	total: 3m 36s	remaining: 25s
1793:	learn: 359.9298626	total: 3m 36s	remaining: 24.9s
1794:	learn: 359.8080338	total: 3m 36s	remaining: 24.8s
1795:	learn: 359.6737698	total: 3m 36s	remaining: 24.6s
1796:	learn: 359.5781406	total: 3m 37s	remaining: 24.5s
1797:	learn: 359.4367816	total: 3m 37s	remaining: 24.4s
1798:	learn: 359.3689128	total: 3m 37s	remaining: 24.3s
1799:	learn: 359.1587348	total: 3m 37s	remaining: 24.2s
1800:	learn: 359.0480003	total: 3m 37s	remaining: 24s
1801:	learn: 358.9458682	total: 3m 37s	remaining: 23.9s
1802:	learn: 358.8829484	total: 3m 37s	remaining: 23

1933:	learn: 344.8937221	total: 3m 54s	remaining: 8s
1934:	learn: 344.7649287	total: 3m 54s	remaining: 7.88s
1935:	learn: 344.6318570	total: 3m 54s	remaining: 7.76s
1936:	learn: 344.5074871	total: 3m 54s	remaining: 7.64s
1937:	learn: 344.4539986	total: 3m 55s	remaining: 7.52s
1938:	learn: 344.3302516	total: 3m 55s	remaining: 7.4s
1939:	learn: 344.2540187	total: 3m 55s	remaining: 7.28s
1940:	learn: 344.2066389	total: 3m 55s	remaining: 7.16s
1941:	learn: 344.1586258	total: 3m 55s	remaining: 7.04s
1942:	learn: 344.0824560	total: 3m 55s	remaining: 6.91s
1943:	learn: 343.9562517	total: 3m 55s	remaining: 6.79s
1944:	learn: 343.8476725	total: 3m 55s	remaining: 6.67s
1945:	learn: 343.7203290	total: 3m 56s	remaining: 6.55s
1946:	learn: 343.6417517	total: 3m 56s	remaining: 6.43s
1947:	learn: 343.5611673	total: 3m 56s	remaining: 6.31s
1948:	learn: 343.4900988	total: 3m 56s	remaining: 6.19s
1949:	learn: 343.3295329	total: 3m 56s	remaining: 6.07s
1950:	learn: 343.2069236	total: 3m 56s	remaining: 5.

In [10]:
preds = model.predict(test_x)

In [11]:
print(preds[:30])

[3696.99690824 1180.81580798 2871.18136001 3369.71254556 3032.05351681
 3321.47132932 3629.02539374 3426.75342892  169.70466207 3308.64679171
 3313.0086791  3056.15538985 2872.91366326 5154.87236281 5715.12555807
  148.40659821 4397.52389337 4356.40047292 4775.72231558 4929.57855534
 4739.29169336 5744.2392191   262.38312447 5400.32793843 6180.6371987
 5242.57458176 5561.1909601  4854.46373884 4396.54760901 1280.4909203 ]


In [12]:
submission = pd.read_csv(os.path.join(file_path,"sample_submission.csv"))
submission

,ID,answer
0,TG_A_J_20230304,0
1,TG_A_J_20230305,0
2,TG_A_J_20230306,0
3,TG_A_J_20230307,0
4,TG_A_J_20230308,0
...,...,...
1087,RD_F_J_20230327,0
1088,RD_F_J_20230328,0
1089,RD_F_J_20230329,0
1090,RD_F_J_20230330,0


In [13]:
holiday_test = ["20230305","20230312","20230319","20230326"]
submission['answer'] = preds
submission.loc[submission['ID'].str.split("_").str[3].isin(holiday_test),"answer"] = 0
submission

,ID,answer
0,TG_A_J_20230304,3696.996908
1,TG_A_J_20230305,0.000000
2,TG_A_J_20230306,2871.181360
3,TG_A_J_20230307,3369.712546
4,TG_A_J_20230308,3032.053517
...,...,...
1087,RD_F_J_20230327,487.976460
1088,RD_F_J_20230328,601.121601
1089,RD_F_J_20230329,746.615622
1090,RD_F_J_20230330,621.246553


In [14]:
submission.to_csv(os.path.join(sub_path,"baseline_submission14.csv"), index=False)